In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 7
fig_height = 5
fig_format = 'retina'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'/Users/mpiekenbrock/simplextree-py/docs_src/pages':
  os.chdir(r'/Users/mpiekenbrock/simplextree-py/docs_src/pages')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


{"/Users/mpiekenbrock/opt/miniconda3/envs/simplextree/lib/python3.9/importlib/_bootstrap.py": 1688592098.220885, "/Users/mpiekenbrock/opt/miniconda3/envs/simplextree/lib/python3.9/importlib/_bootstrap_external.py": 1688592098.223086, "/Users/mpiekenbrock/opt/miniconda3/envs/simplextree/lib/python3.9/codecs.py": 1688592096.019708, "/Users/mpiekenbrock/opt/miniconda3/envs/simplextree/lib/python3.9/encodings/aliases.py": 1688592098.717748, "/Users/mpiekenbrock/opt/miniconda3/envs/simplextree/lib/python3.9/encodings/__init__.py": 1688592098.715795, "/Users/mpiekenbrock/opt/miniconda3/envs/simplextree/lib/python3.9/encodings/utf_8.py": 1688592098.947974, "/Users/mpiekenbrock/opt/miniconda3/envs/simplextree/lib/python3.9/encodings/latin_1.py": 1688592098.886754, "/Users/mpiekenbrock/opt/miniconda3/envs/simplextree/lib/python3.9/abc.py": 1688592095.983326, "/Users/mpiekenbrock/opt/miniconda3/envs/simplextree/lib/python3.9/io.py": 1688592096.09689, "/Users/mpiekenbrock/opt/miniconda3/envs/simp

In [2]:
#| echo: true
#| lst-cap: The SimplexTree class provides light wrapper around the extension module
#| output: true
#| code-summary: SimplexTree constructor
from simplextree import SimplexTree 
st = SimplexTree([[0,1,2], [0,1], [4,5]]) 
print(st) 

Simplex Tree with (5, 4, 1) (0, 1, 2)-simplices


In [3]:
#| echo: true
#| output: true
#| code-summary: Batch operations
st.insert([[1,4], [1,5], [6]])
print(st)

Simplex Tree with (6, 6, 1) (0, 1, 2)-simplices


In [4]:
st.remove([[6]])
print(st)

Simplex Tree with (5, 6, 1) (0, 1, 2)-simplices


In [5]:
print(st.find([[6], [0,1]]))

[False  True]


In [6]:
#| echo: true
#| output: true
#| code-summary: Complex collections 
print(st.simplices())

[[0], [1], [2], [4], [5], [0, 1], [0, 2], [1, 2], [1, 4], [1, 5], [4, 5], [0, 1, 2]]


In [7]:
#| echo: true
#| output: true
print(st.simplices(p=1)) 

[[0, 1], [0, 2], [1, 2], [1, 4], [1, 5], [4, 5]]


In [8]:
#| echo: true
#| output: true
print([0,1,2] in st)
print([len(simplex) for simplex in st])
print(len(st))

True
[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3]
12


In [9]:
print("Cofaces([1]): " + str(st.cofaces([1])))

Cofaces([1]): [[1], [1, 2], [1, 4], [1, 5], [0, 1], [0, 1, 2]]


In [10]:
print("Maximal: " + str(st.maximal()))

Maximal: [[0, 1, 2], [1, 4], [1, 5], [4, 5]]


In [11]:
st.n_simplices, st.dimension, st.vertices, st.connected_components

(array([5, 6, 1], dtype=uint64), 2, [0, 1, 2, 4, 5], [1, 1, 1, 1, 1])

In [12]:
import numpy as np 
all(np.all(st.triangles == np.array(st.simplices(p=2)), axis=0))

True

In [13]:
st.insert([[1,4]]) 
st.expand(2)       
print(st)

Simplex Tree with (5, 6, 2) (0, 1, 2)-simplices


In [14]:
st.print_tree()

0 (h = 2): .( 1 2 )..( 2 )


1 (h = 2): .( 2 4 5 )..( 5 )


2 (h = 0): 


4 (h = 1): .( 5 )


5 (h = 0): 
